# Finding the best Neighbourhood to live in Vancouver

### In this project, I am going to find out where is best to live in Vancouver for a just married couple, they are planning to move to Vancouver, but dont know the different Neighborhoods.

### I will be downloding all data from Foursqare and compare the different venues in each neigborhood. then , I will be taking the mean rating from each category and average the mean ratings to find out which is the best neighbourhood.

## 1. Import Libraries

In [1]:
#!pip install geopy
!pip install bs4

In [2]:
import requests  # this module helps us to download a web page
import pandas as pd
from bs4 import BeautifulSoup # this module helps in web scrapping.
import folium
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

## Reading Vancouber Neighbourhoods

In [20]:
url = 'https://www.zipcodesonline.com/2020/07/vancouver-postal-code.html'
raw_table=pd.read_html(url, flavor='bs4')[1]
raw_table.columns=['SL','Neighbourhood','PostCode','City']
raw_table= raw_table.dropna()
raw_table.reset_index(inplace=True, drop=True)

In [44]:
raw_table.head()

,SL,Neighbourhood,PostCode,City
0,SL. NO.,NEIGHBOURHOOD / LOCALITY,POSTAL CODES,CITY
1,1,Bentall Centre,V7X,Vancouver
2,2,Canada Place,V6C,Vancouver
3,3,Central Kitsilano,V6K,Vancouver
4,4,Chaldecutt,V6S,Vancouver


In [57]:
df = pd.DataFrame(columns=["Postal Code","Neighbourhood","Latitude","Longitude"])
for i in raw_table.index:
    try:
        address = raw_table.loc[i,'Neighbourhood']+', Vancouver, BC'
        geolocator = Nominatim(user_agent="explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        df = df.append({"Postal Code": raw_table.loc[i,'PostCode'],"Latitude":  latitude,"Longitude":  longitude, "Neighbourhood": raw_table.loc[i,'Neighbourhood']}, ignore_index=True)
    except:
        x=1
df.head()

,Postal Code,Neighbourhood,Latitude,Longitude
0,V7X,Bentall Centre,49.286234,-123.119028
1,V6C,Canada Place,49.288404,-123.111966
2,V6K,Central Kitsilano,49.269410,-123.155267
3,V6A,Chinatown,49.279981,-123.104089
4,V6C,Coal Harbour,49.290375,-123.129281


In [111]:
# mapping the neighbourhoods.
map_neigh = folium.Map(location=[df['Latitude'][0], df['Longitude'][0]], zoom_start=11)
# add markers to the map
for lat, lon, poi in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=poi,
        fill=True,
        fill_opacity=0.7).add_to(map_neigh)

In [ ]:
map_neigh

## 2. Read Vancouver Venues Data

In [62]:
#Foursquare credentials
CLIENT_ID = 'LVQZMVH2XZMPQLRMMR2TZ04O0LQ1TYZRMPGA54UD5X1EVYZU' # your Foursquare ID
CLIENT_SECRET = 'KCLBLZIS33ZJDINHE0FW0NXJ3LSRQYPIGWRMA0DVTGJSZYOP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [64]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [176]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):  
    LIMIT = 100
    venues_list=pd.DataFrame(columns=['Neighbourhood','name', 'categories', 'lat', 'lng','postalCode','id'])
    for name, lat, lng in zip(names, latitudes, longitudes):  
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        try:    
            # make the GET request
            results = requests.get(url).json()
            venues = results['response']['groups'][0]['items'] 
            normalized_venues = pd.json_normalize(venues)
            filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.postalCode','venue.id']
            normalized_venues =normalized_venues.loc[:, filtered_columns]
            normalized_venues['venue.categories'] = normalized_venues.apply(get_category_type, axis=1)
            normalized_venues.columns = [col.split(".")[-1] for col in normalized_venues.columns]
            normalized_venues['Neighbourhood'] = name
            # append new data
            venues_list = pd.concat([venues_list,normalized_venues],ignore_index=True)
        except:
            print("error")
    return(venues_list)

In [177]:
neigh_venues = getNearbyVenues(names=df['Neighbourhood'],
                               latitudes=df['Latitude'],
                               longitudes=df['Longitude']
                               )

error
error


In [182]:
#remove_duplicates
neigh_venues.drop_duplicates(subset=['id'],inplace=True)
neigh_venues.shape

(1202, 7)

In [181]:
#save vanues to csv
neigh_venues.to_csv('Vancouver_venues.csv',index=False)

In [184]:
#As there is a limit for getting the ratings for free accounts, I am doing the analysis with only the restaurants.
neigh_venues_restaurant=neigh_venues[neigh_venues.categories.str.contains("Restaurant")]
neigh_venues_restaurant.shape

(338, 7)

In [189]:
#get ratings of restaurants
venues_l = pd.DataFrame(columns=['id','rating','ratingSignals','tipCount','LikesCount','PhotosCount','TipsCount'])
for row in neigh_venues_restaurant.index:
    try:
        VENUE_ID=neigh_venues_restaurant['id'][row]
        # create URL
        url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            VENUE_ID,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
        url # display URL
        results2 = requests.get(url).json()
        venue_stats = pd.json_normalize(results2["response"]['venue']) # flatten JSON
        # filter columns
        #filtered_columns = ['id','rating','ratingSignals','stats.tipCount','price.tier','likes.count','photos.count','tips.count']
        filtered_columns = ['id','rating','ratingSignals','stats.tipCount','likes.count','photos.count','tips.count']
        venue_stats =venue_stats.loc[:, filtered_columns]

        # clean columns
        venue_stats.columns = [col.split(".")[-1] for col in venue_stats.columns]
        #venue_stats.columns = ['id','rating','ratingSignals','tipCount','PriceTier','LikesCount','PhotosCount','TipsCount']
        venue_stats.columns = ['id','rating','ratingSignals','tipCount','LikesCount','PhotosCount','TipsCount']
        venues_l = pd.concat([venues_l,venue_stats], sort=False)
    except:
        continue
venues_l.shape


error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error


(312, 7)

In [190]:
venues_merged=pd.merge(neigh_venues_restaurant, venues_l, on="id")
venues_merged.shape

(312, 13)

In [191]:
venues_merged.head()

,Neighbourhood,name,categories,lat,lng,postalCode,id,rating,ratingSignals,tipCount,LikesCount,PhotosCount,TipsCount
0,Bentall Centre,Cactus Club Cafe,American Restaurant,49.285699,-123.119249,V6C 0A8,4aaa9a0ef964a5208d5620e3,8.2,325,75,216,411,75
1,Bentall Centre,Nightingale,Restaurant,49.287343,-123.117674,V6E 0C4,572e921e498ea744e2e8a8b0,8.2,114,20,84,86,20
2,Bentall Centre,The Keg Steakhouse + Bar - Dunsmuir,Restaurant,49.283438,-123.116363,V6B 1N3,5170656ae4b029a050dd8a49,8.8,203,33,158,123,33
3,Bentall Centre,Cactus Club Cafe,American Restaurant,49.289468,-123.117671,V6C 0C3,4d320c5498336dcb18401ff0,8.5,617,98,466,698,98
4,Bentall Centre,Joe Fortes Seafood & Chop House,Seafood Restaurant,49.285017,-123.124410,V6E 3V5,4ab59f7cf964a520a97520e3,9.0,521,118,385,781,118


In [207]:
#Convert to float
venues_merged['LikesCount'] = venues_merged['LikesCount'].astype(float)
venues_merged['TipsCount'] = venues_merged['TipsCount'].astype(float)

In [209]:
#venues with best ratings
venues_merged.groupby(by="Neighbourhood", dropna=False).mean().sort_values(by=['rating'],ascending=False).head()

,lat,lng,rating,LikesCount,TipsCount
Neighbourhood,,,,,
Northwest West End,49.284070,-123.125610,8.766667,112.666667,47.333333
Northeast Downtown,49.276847,-123.113717,8.600000,32.000000,13.000000
Chinatown,49.281249,-123.104528,8.080000,90.700000,32.133333
Bentall Centre,49.284938,-123.119313,8.068750,125.625000,40.062500
Canada Place,49.286114,-123.113201,8.040000,141.600000,64.800000


In [210]:
#venues with more likes
venues_merged.groupby(by="Neighbourhood", dropna=False).mean().sort_values(by=['LikesCount'],ascending=False).head()

,lat,lng,rating,LikesCount,TipsCount
Neighbourhood,,,,,
Canada Place,49.286114,-123.113201,8.040000,141.600000,64.800000
Bentall Centre,49.284938,-123.119313,8.068750,125.625000,40.062500
Northwest West End,49.284070,-123.125610,8.766667,112.666667,47.333333
Chinatown,49.281249,-123.104528,8.080000,90.700000,32.133333
Gastown,49.283587,-123.108127,7.828571,86.428571,153.857143


In [208]:
#venues with more tips
venues_merged.groupby(by="Neighbourhood", dropna=False).mean().sort_values(by=['TipsCount'],ascending=False).head()

,lat,lng,rating,LikesCount,TipsCount
Neighbourhood,,,,,
Gastown,49.283587,-123.108127,7.828571,86.428571,153.857143
Canada Place,49.286114,-123.113201,8.040000,141.600000,64.800000
Northwest West End,49.284070,-123.125610,8.766667,112.666667,47.333333
Bentall Centre,49.284938,-123.119313,8.068750,125.625000,40.062500
Yaletown,49.276608,-123.120822,8.022857,77.714286,33.371429


## Conclusion: The best Neighbourhood to live in Vancouver based on the restaurants parameters is the Northwest West End.